In [1]:
import torch
import numpy as np
import nltk
#nltk.download('punkt_tab')

In [19]:
from datasets import load_from_disk
dataset = load_from_disk("./resources/data/train_dataset/")
train_dataset = dataset['train']
val_dataset = dataset['validation']

In [2]:
from transformers import Trainer, EvalPrediction, TrainingArguments, DataCollatorWithPadding, EarlyStoppingCallback, AutoModelForSeq2SeqLM
from datasets import load_metric

/data/ephemeral/home/mrc-nlp-08/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# BART

In [12]:
# Load model directly
from transformers import AutoTokenizer, BartForConditionalGeneration

tokenizer = AutoTokenizer.from_pretrained("gogamza/kobart-base-v2")
model = BartForConditionalGeneration.from_pretrained("gogamza/kobart-base-v2")

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


In [7]:
train_dataset[0]

{'title': '미국 상원',
 'context': '미국 상의원 또는 미국 상원(United States Senate)은 양원제인 미국 의회의 상원이다.\\n\\n미국 부통령이 상원의장이 된다. 각 주당 2명의 상원의원이 선출되어 100명의 상원의원으로 구성되어 있다. 임기는 6년이며, 2년마다 50개주 중 1/3씩 상원의원을 새로 선출하여 연방에 보낸다.\\n\\n미국 상원은 미국 하원과는 다르게 미국 대통령을 수반으로 하는 미국 연방 행정부에 각종 동의를 하는 기관이다. 하원이 세금과 경제에 대한 권한, 대통령을 포함한 대다수의 공무원을 파면할 권한을 갖고 있는 국민을 대표하는 기관인 반면 상원은 미국의 주를 대표한다. 즉 캘리포니아주, 일리노이주 같이 주 정부와 주 의회를 대표하는 기관이다. 그로 인하여 군대의 파병, 관료의 임명에 대한 동의, 외국 조약에 대한 승인 등 신속을 요하는 권한은 모두 상원에게만 있다. 그리고 하원에 대한 견제 역할(하원의 법안을 거부할 권한 등)을 담당한다. 2년의 임기로 인하여 급진적일 수밖에 없는 하원은 지나치게 급진적인 법안을 만들기 쉽다. 대표적인 예로 건강보험 개혁 당시 하원이 미국 연방 행정부에게 퍼블릭 옵션(공공건강보험기관)의 조항이 있는 반면 상원의 경우 하원안이 지나치게 세금이 많이 든다는 이유로 퍼블릭 옵션 조항을 제외하고 비영리건강보험기관이나 보험회사가 담당하도록 한 것이다. 이 경우처럼 상원은 하원이나 내각책임제가 빠지기 쉬운 국가들의 국회처럼 걸핏하면 발생하는 의회의 비정상적인 사태를 방지하는 기관이다. 상원은 급박한 처리사항의 경우가 아니면 법안을 먼저 내는 경우가 드물고 하원이 만든 법안을 수정하여 다시 하원에 되돌려보낸다. 이러한 방식으로 단원제가 빠지기 쉬운 함정을 미리 방지하는 것이다.날짜=2017-02-05',
 'question': '대통령을 포함한 미국의 행정부 견제권을 갖는 국가 기관은?',
 'id': 'mrc-1-000067',
 'answers': {'answer_start'

In [8]:
len(train_dataset), len(val_dataset)

(3952, 240)

In [20]:
# Train preprocessing / 전처리를 진행합니다.
def prepare_features(examples):
    # truncation과 padding(length가 짧을때만)을 통해 toknization을 진행하며, stride를 이용하여 overflow를 유지합니다.
    # 각 example들은 이전의 context와 조금씩 겹치게됩니다.
    labels = [answers['text'][0] for answers in examples['answers']]
    tokenized_examples = tokenizer(
        text=examples['question'],
        text_pair=examples['context'],
        text_target=labels,
        padding="max_length",
        truncation="only_second",    # context만 truncate하기
        max_length=512,
        stride=128, # 이전 chunk와 overlap되는 부분을 두어 긴 문서를 처리할 때 유용. 모델이 더 나은 임베딩을 하도록 도와 QA에 유용.
        return_overflowing_tokens=True,
    )
    del tokenized_examples['token_type_ids']
    # labels 확장
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    tokenized_examples["labels"] = [tokenized_examples['labels'][i] for i in sample_mapping]
    
    return tokenized_examples

train_dataset = train_dataset.map(
            prepare_features,
            batched=True,
            num_proc=None,
            remove_columns=train_dataset.column_names,
            load_from_cache_file=not False)

val_dataset = val_dataset.map(
            prepare_features,
            batched=True,
            num_proc=None,
            remove_columns=val_dataset.column_names,
            load_from_cache_file=not False)

Map: 100%|██████████| 240/240 [00:00<00:00, 1507.18 examples/s]


In [21]:
train_dataset, val_dataset

(Dataset({
     features: ['input_ids', 'attention_mask', 'labels'],
     num_rows: 5107
 }),
 Dataset({
     features: ['input_ids', 'attention_mask', 'labels'],
     num_rows: 307
 }))

In [24]:
metric = load_metric("squad")

def compute_metrics(p: EvalPrediction): #EvalPrediction 구조 | predictions: 모델의 예측값, label_ids: 실제 정답 레이블
    predictions = p.predictions
    references = p.label_ids
    
    if np.array(predictions).ndim == 3:
        predictions = np.argmax(predictions, axis=-1)
        
    predictions = [{'prediction_text': pred, 'id': str(i)} for i, pred in enumerate(predictions)]
    references = [{'answers': {'answer_start': [], 'text': [label]}, 'id': str(i)} for i, label in enumerate(references)]

    result = metric.compute(predictions=predictions, references=references)
    result['eval_exact_match'] = result['exact_match']
    del result['exact_match']
    result['eval_f1'] = result['f1']
    del result['f1']
    return result

training_args = TrainingArguments(
    output_dir="./ckpt/bart",num_train_epochs = 25,
    save_strategy='epoch', evaluation_strategy="epoch",
    remove_unused_columns=False, optim="adamw_hf",
    per_device_train_batch_size=16, weight_decay=0.01,
    save_total_limit = 1, load_best_model_at_end = True)

    #warmup_steps=1000,

data_collator = DataCollatorWithPadding(tokenizer, pad_to_multiple_of=8 if training_args.fp16 else None)

# Trainer 초기화
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
    #callbacks = [EarlyStoppingCallback(early_stopping_patience=10)]
)

/data/ephemeral/home/mrc-nlp-08/.venv/lib/python3.10/site-packages/transformers/training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [25]:
torch.cuda.empty_cache()
trainer.train()

/data/ephemeral/home/mrc-nlp-08/.venv/lib/python3.10/site-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,No log,0.013695
2,0.007500,0.014684
3,0.007500,0.015471
4,0.002600,0.016835
5,0.000900,0.016778
6,0.000900,0.016847
7,0.000500,0.017352
8,0.000300,0.018456


KeyboardInterrupt: 

In [ ]:
trainer.train(resume_from_checkpoint=True)

In [26]:
def inference(context, question):
    # 모델을 평가 모드로 설정
    trainer.model.eval()


    # 입력 텍스트를 토크나이즈
    inputs = tokenizer(
            text=question,
            text_pair=context,
            padding="max_length",
            truncation="only_second",    # context만 truncate하기
            max_length=384,
            stride=128, # 이전 chunk와 overlap되는 부분을 두어 긴 문서를 처리할 때 유용. 모델이 더 나은 임베딩을 하도록 도와 QA에 유용.
            return_tensors='pt'
        )
    del inputs['token_type_ids']

    # GPU 사용 시 입력을 GPU로 이동
    inputs = {k: torch.tensor(v).to(trainer.model.device) for k, v in inputs.items()}

    outputs = model.generate(**inputs, max_length=30, num_beams=5)
    '''# inference 수행
    with torch.no_grad():
        outputs = trainer.model(**inputs)'''
    # 결과 처리
    #predictions = outputs.argmax(-1)
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return generated_text

In [27]:
context = ''''ㅋㅋ'은 일반적으로 웃음소리를 표현할 때 사용된다. 통신체에서는 웃는 모습을 직접적으로 보여줄 수 없기 때문에 의성어를 사용하는 경우가 많은데 그 중에서 '킥킥', '큭큭'을 초성체로 바꾸어 사용한 것이 'ㅋㅋ'에 해당한다. 이에 대해 2001년 연구에서는 "자음만 가지고 표시하여 통신상의 재미를 더하기 위한 방법으로 사용되고 있다"고 해석한 바 있다. \n\nㅋ자를 웃음소리를 의미하는 것으로 사용할 때, 'ㅋ'자 한 개에서 'ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ...'하는 식으로 열 타 이상 한 줄 넘게까지 쓰이기도 한다 ㅋ자를 얼마나 많이 썼느냐에 따라 그 느낌과 의미는 달라진다. 신조어를 전문으로 연구하는 이재현 문화평론가는 《한국일보》의 기고글에서 그 종류와 느낌이 대략 다음과 같다고 보았다\n* ㅋ - 일반적으로 무심하게 동의하려는 상황에서 쓰인다. 다른 말끝에 붙였을 때, 이를테면 '그래ㅋ', '좋네ㅋ', '올ㅋ' 등은 웃음 자체라기보다는 입꼬리가 아주 살짝 올라간 상황이라는 느낌을 준다. 의성어로는 "큭" 혹은 "킥"으로 해석될 수 있다. 한편 '뭐해?ㅋ', '미안ㅋ'의 경우 'ㅋ'는 상투적인 군말의 역할을 한다.\n* ㅋㅋ - 위의 것보다 ㅋ자가 하나가 더 많으나 오히려 형식적이라는 느낌을 준다. 좋게는 '그렇군'을 뜻하거나, 조금 나쁘게는 대화상에서 의례적으로 추임새를 넣고 있다는 느낌을 준다. 'ㅋㅋ'은 미혼남녀들이 가장 싫어하는 성의 없는 메신저 말투로 선정되기도 했다. 다만 회사 상사에게 대답하는 경우 같은 공적인 상황에서는 '네ㅋㅋ'와 같은 대답은 가벼워 보인다는 조사도 있었다. \n* ㅋㅋㅋ - 비교적 중립적이면서도 가장 많이 사용되는 용법이다. 웃는 감정을 그대로 표현한다는 뉘앙스를 갖는다.\n* ㅋㅋㅋㅋ... - 여기서부터는 "정말 웃긴다"라는 반응을 나름 정성들여서 표현하는 느낌을 준다. 네 개부터 그 이상은 의미의 함축과 정서의 강도가 거의 같으며, 이런 점에서 'ㅋㅋㅋㅋ'는 실질적으로 웃긴다는 것을 표현하는 최소 조건에 해당한다.'''
question = '웃는 감정을 그대로 표현한다는 뉘앙스를 갖는 ㅋ의 용법은?'
generated_text = inference(context, question)
print(f"Generated text: {generated_text}")

Generated text: 'ᄏᄏᄏ'


/tmp/ipykernel_474890/3863879182.py:19: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  inputs = {k: torch.tensor(v).to(trainer.model.device) for k, v in inputs.items()}


In [28]:
context = '''냉대 동계 소우 기후(冷带冬季少雨氣候)는 쾨펜의 기후 구분에서 냉대 기후에 속하며, 기호는 Dwa, Dwb, Dwc, Dwd로, D는 냉대, w는 동계건조(wintertrocken)을 가리킨다. 트란스 바이칼 기후 라고도 한다. 여름에는 온도가 비교적 높고, 겨울에는 맑은 날씨가 지속되고 약한 바람이 분다. 복사냉각 때문에 매우 쌀쌀한 것이 특징이다. 또 여름에는 계절풍의 영향으로 비가 많이 내린다. 겨울은 여름에 비해 매우 긴 편이다. 기온의 연교차가 매우 크게 나타나는 대륙성 기후가 짙게 나타난다.\n\n냉대 동계 소우 기후는 매우 추운 기후로, 농작물을 재배할 수 있는 곳과 재배 수종 및 수가 제한되어 있다. 수수, 옥수수, 봄밀 등을 재배한다. 북부는 타이가 기후와 같이 타이가가 많이 나타나 임업활동을 한다. 숲 또한 냉대 습윤 기후와 비슷하여 남부에는 침엽수와 활엽수가 함께 나타나는 혼합림이, 북부에는 타이가(침엽수림)이 잘 나타난다. 북부는 추위에도 잘 견디는 가늘고 뾰족한 잎을 가진 침엽수가 자라기 유리하기 때문에 타이가가, 남부에는 북부보다 따뜻하여 혼합림이 잘 나타난다. 남부 지역의 극소수에서만 벼 재배가 가능하다.\n\n토양은 주로 포드졸이 나타나고, 혼합림에서는 갈색 삼림토가 나타나는데, 포드졸은 유기물이 분해되지는 않았으나, 소금기를 잘 머금지 않고 침엽수림의 특성 때문에 농작물을 재배하는 데에는 적합하지 않다. 그에 반해 갈색 삼림토는 물을 잘 흡수하고 포드졸에 비해 약산성을 띠기 때문에 농작물 재배와 목축에 알맞다.\n\n이런 냉대 동계 소우 기후가 나타나는 곳으로는 시베리아 내륙 동부, 중국 화베이 동부 및 둥베이, 대한민국 강원도 내륙 지역과 경기도 및 산악지대, 조선민주주의인민공화국 대부분, 캐나다 및 미국, 네팔의 일부 지역 등이 있으며, 특히 시베리아 동부는 세계의 한극을 만들어 낸다.\n\n그래서 이 기후가 냉대 기후의 탈을 쓴 사바나 기후라고 봐야 하는 수준에 이르게 되며, 겨울철에는 강수량이 적어져서 그렇게 건조한 만큼 불씨가 쉽게 일어나는 특성상 산불도 물론 일어나기가 쉬운 기후 대역에 속한다.\n\n앞으로 온난화 여파에 따라 이 기후가 사라질 가능성이 있는 지역도 생길 수도 있다. 단적인 예로 대한민국의 수도권, 강원도 등은 현재 Dwa로 분류되나 온대 기후로 바뀔 가능성이 있다.'''
question = '냉대 동계 소우 기후의 w는 무엇의 약자인가?'
generated_text = inference(context, question)
print(f"Generated text: {generated_text}")

Generated text: 동계건조


/tmp/ipykernel_474890/3863879182.py:19: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  inputs = {k: torch.tensor(v).to(trainer.model.device) for k, v in inputs.items()}


In [29]:
context = '''한국의 웹툰.\n\n곽백수가 네이버에서 연재한 웹툰. 회사 생활의 이야기를 주축으로 하는 직장 만화로, 작가의 이전 흥행작인《트라우마》에서도 익히 이름을 알렸던 가우스 그룹#s-1.3이 아예 메인으로 다뤄진다. 따라서, 어떤 의미에서는 트라우마의 스핀오프작이라고도 볼수 있는 셈. 물론, 가우스 그룹의 막나가는 센스는 여전하다. 초기에는 《사이좋은 가우스전자》라는 이름이었지만 어느 순간 '사이좋은'이 빠지고 그냥 《가우스전자》가 되었다.\n\n기본적으로《트라우마》와 비슷한 분위기의 개그물이지만 블랙 코미디의 성격이 강하다. 게다가 주제가 주제인지라 회사원 입장에서 공감할 내용들이 많으며, 그런 독자들의 반응은 대개 '내가 웃어도 웃는 게 아니야.' 로 통일되어 있다. 그래도 불편한 진실과 블랙 코미디 사이에서 블랙 코미디 쪽으로 줄타기를 잘 하고 있다. 또한 브리핑 에피소드들을 보면 현 업계의 시류를 잘 집고 있어 작가가 에피소드 소재에 대한 준비를 열심히 한 듯한 느낌이 난다. 그렇기에 특히 회사원들, 즉 어느 정도 연령층이 높은 독자들에게 평이 상당히 좋다. 그래도 상대적으로 악역 포지션인 인물들이 있고 그렇지 않은 인물들이 있지만, 현실 기업과는 비교도 할 수 없을만큼의 훈훈함이 특징.'''
question = '《가우스전자》의 원래 제목은 무엇이었나?'
generated_text = inference(question, context)
print(f"Generated text: {generated_text}")

Generated text: 《사이좋은 가우스전자》


/tmp/ipykernel_474890/3863879182.py:19: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  inputs = {k: torch.tensor(v).to(trainer.model.device) for k, v in inputs.items()}


In [37]:
def inference(context, question, answer):
    trainer.model.eval()
    # 입력 텍스트를 토크나이즈
    inputs = tokenizer(
            text=question,
            text_pair=context,
            padding="max_length",
            truncation=True,    # context만 truncate하기
            max_length=384,
            stride=128, # 이전 chunk와 overlap되는 부분을 두어 긴 문서를 처리할 때 유용. 모델이 더 나은 임베딩을 하도록 도와 QA에 유용.
            return_tensors='pt'
        )
    del inputs['token_type_ids']

    # GPU 사용 시 입력을 GPU로 이동
    inputs = {k: v.to('cuda') for k, v in inputs.items()}

    outputs = model.generate(**inputs, max_new_tokens=30, num_beams=5)

    # 결과 처리
    #predictions = outputs.argmax(-1)
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    print(f"pred : {generated_text}")
    print(f"answer : {answer}")

In [38]:
from datasets import Dataset
datasets = load_from_disk('./resources/data/train_dataset')
valid_dataset = datasets['validation']

valid_dataset = valid_dataset.shuffle(seed=104)
answers = [answer['text'] for answer in valid_dataset['answers']]
test = Dataset.from_dict({'context':valid_dataset['context'][:10], 'question':valid_dataset['question'][:10], 'answers':valid_dataset['answers'][:10]})

# pretrained model 과 tokenizer를 불러오기

assert model.config.vocab_size == len(tokenizer), "Model and tokenizer vocab sizes do not match!"
for data in test:
    inference(data['context'], data['question'], data['answers'])

pred : 높은 벽들로 감싼 형태
answer : {'answer_start': [789], 'text': ["'달빛 정원'(Moonlight Garden)"]}
pred : 미타케 성
answer : {'answer_start': [304], 'text': ['미타케성']}
pred : 초나라왕
answer : {'answer_start': [1084], 'text': ['제 양왕']}
pred : 우천
answer : {'answer_start': [406], 'text': ['우천']}
pred : 태화관
answer : {'answer_start': [115], 'text': ['태화관(서울시 종로구 인사동 소재)']}
pred : 김수환
answer : {'answer_start': [0], 'text': ['김수환 추기경']}
pred : 대집회
answer : {'answer_start': [1029], 'text': ['데코행진']}
pred : 의상대사
answer : {'answer_start': [35], 'text': ['의상대사']}
pred : 릿지당
answer : {'answer_start': [585], 'text': ['메이저 릿지']}
pred : 어리석거나 나쁘다는 식
answer : {'answer_start': [468], 'text': ['공정']}


# T5

In [3]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("KETI-AIR/ke-t5-small")
model = AutoModelForSeq2SeqLM.from_pretrained("KETI-AIR/ke-t5-small", use_fast=False)

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [ ]:
train_dataset[0]

In [21]:
from transformers import (
    DataCollatorForSeq2Seq,
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments
)
max_source_length = 384
max_target_length = 16
padding = "max_length"
preprocessing_num_workers = 12
num_beams = 5
max_train_samples = 3952
max_val_samples = 240
num_train_epochs = 50
train_batch_size = 16
eval_batch_size = 8
learning_rate = 1e-3

In [ ]:
data_collator = DataCollatorForSeq2Seq(
            tokenizer,
            model=model,
        )

# Train preprocessing / 전처리를 진행합니다.
def prepare_features(examples):
    # truncation과 padding(length가 짧을때만)을 통해 toknization을 진행하며, stride를 이용하여 overflow를 유지합니다.
    # 각 example들은 이전의 context와 조금씩 겹치게됩니다.
    inputs = [f'question: {q}  context: {c}' for q, c in zip(examples['question'], examples['context'])]
    targets = [f'{a["text"][0]}' for a in examples['answers']]
    model_inputs = tokenizer(inputs, max_length=max_source_length, padding=padding, truncation=True, return_tensors='pt')

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=max_target_length, padding=padding, truncation=True, return_tensors='pt')

    model_inputs["labels"] = labels["input_ids"]
    model_inputs["labels"][model_inputs["labels"] == tokenizer.pad_token_id] = -100
        
    return model_inputs

train_dataset = train_dataset.map(
            prepare_features,
            batched=True,
            num_proc=None,
            remove_columns=train_dataset.column_names,
            load_from_cache_file=not False)

print('validation')
val_dataset = val_dataset.map(
            prepare_features,
            batched=True,
            num_proc=None,
            remove_columns=val_dataset.column_names,
            load_from_cache_file=not False)


In [22]:
metric = load_metric('squad')
def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [label.strip() for label in labels]

    preds = ["\n".join(nltk.sent_tokenize(pred)) for pred in preds]
    labels = ["\n".join(nltk.sent_tokenize(label)) for label in labels]

    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]

    preds = np.where(preds != -100, preds, tokenizer.pad_token_id)
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    # decoded_labels is for rouge metric, not used for f1/em metric

    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    formatted_predictions = [{"id": ex['id'], "prediction_text": decoded_preds[i]} for i, ex in enumerate(dataset['validation'].select(range(max_val_samples)))]
    references = [{"id": ex["id"], "answers": ex["answers"]} for ex in dataset['validation'].select(range(max_val_samples))]

    result = metric.compute(predictions=formatted_predictions, references=references)
    return result

In [ ]:
args = Seq2SeqTrainingArguments(
    output_dir='./ckpt/t5',
    do_train=True,
    do_eval=True,
    optim="adamw_hf",
    warmup_steps=1000,
    weight_decay=0.01,
    per_device_train_batch_size=train_batch_size,
    per_device_eval_batch_size=eval_batch_size,
    predict_with_generate=True,
    num_train_epochs=num_train_epochs,
    save_strategy = 'epoch',
    evaluation_strategy = 'epoch',
    save_total_limit = 2,
    logging_strategy = 'epoch',
    logging_dir="./ckpt/t5/logs",
    load_best_model_at_end = True,
    learning_rate=learning_rate,
    remove_unused_columns = True
)

In [25]:
trainer = Seq2SeqTrainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    callbacks = [EarlyStoppingCallback(early_stopping_patience=10)]
)

In [ ]:
torch.cuda.empty_cache()
trainer.train()

In [ ]:
torch.cuda.empty_cache()
trainer.train(resume_from_checkpoint=True)

In [27]:
def generate_answer(question, context):
    inputs = f'question: {question}  context: {context} </s>'
    sample = tokenizer(inputs, max_length=max_source_length, padding=padding, truncation=True, return_tensors='pt')
    sample = sample.to("cuda:0")
    outputs = model.generate(**sample, max_length=max_target_length, num_beams=num_beams)
    pred = tokenizer.decode(outputs[0], skip_special_tokens=True)
    pred = "\n".join(nltk.sent_tokenize(pred))
    return pred

In [ ]:
context = ''''ㅋㅋ'은 일반적으로 웃음소리를 표현할 때 사용된다. 통신체에서는 웃는 모습을 직접적으로 보여줄 수 없기 때문에 의성어를 사용하는 경우가 많은데 그 중에서 '킥킥', '큭큭'을 초성체로 바꾸어 사용한 것이 'ㅋㅋ'에 해당한다. 이에 대해 2001년 연구에서는 "자음만 가지고 표시하여 통신상의 재미를 더하기 위한 방법으로 사용되고 있다"고 해석한 바 있다. \n\nㅋ자를 웃음소리를 의미하는 것으로 사용할 때, 'ㅋ'자 한 개에서 'ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ...'하는 식으로 열 타 이상 한 줄 넘게까지 쓰이기도 한다 ㅋ자를 얼마나 많이 썼느냐에 따라 그 느낌과 의미는 달라진다. 신조어를 전문으로 연구하는 이재현 문화평론가는 《한국일보》의 기고글에서 그 종류와 느낌이 대략 다음과 같다고 보았다\n* ㅋ - 일반적으로 무심하게 동의하려는 상황에서 쓰인다. 다른 말끝에 붙였을 때, 이를테면 '그래ㅋ', '좋네ㅋ', '올ㅋ' 등은 웃음 자체라기보다는 입꼬리가 아주 살짝 올라간 상황이라는 느낌을 준다. 의성어로는 "큭" 혹은 "킥"으로 해석될 수 있다. 한편 '뭐해?ㅋ', '미안ㅋ'의 경우 'ㅋ'는 상투적인 군말의 역할을 한다.\n* ㅋㅋ - 위의 것보다 ㅋ자가 하나가 더 많으나 오히려 형식적이라는 느낌을 준다. 좋게는 '그렇군'을 뜻하거나, 조금 나쁘게는 대화상에서 의례적으로 추임새를 넣고 있다는 느낌을 준다. 'ㅋㅋ'은 미혼남녀들이 가장 싫어하는 성의 없는 메신저 말투로 선정되기도 했다. 다만 회사 상사에게 대답하는 경우 같은 공적인 상황에서는 '네ㅋㅋ'와 같은 대답은 가벼워 보인다는 조사도 있었다. \n* ㅋㅋㅋ - 비교적 중립적이면서도 가장 많이 사용되는 용법이다. 웃는 감정을 그대로 표현한다는 뉘앙스를 갖는다.\n* ㅋㅋㅋㅋ... - 여기서부터는 "정말 웃긴다"라는 반응을 나름 정성들여서 표현하는 느낌을 준다. 네 개부터 그 이상은 의미의 함축과 정서의 강도가 거의 같으며, 이런 점에서 'ㅋㅋㅋㅋ'는 실질적으로 웃긴다는 것을 표현하는 최소 조건에 해당한다.'''
question = '웃는 감정을 그대로 표현한다는 뉘앙스를 갖는 ㅋ의 용법은?'
generated_text = generate_answer(question, context)
print(f"Generated text: {generated_text}")

In [ ]:
context = '''냉대 동계 소우 기후(冷带冬季少雨氣候)는 쾨펜의 기후 구분에서 냉대 기후에 속하며, 기호는 Dwa, Dwb, Dwc, Dwd로, D는 냉대, w는 동계건조(wintertrocken)을 가리킨다. 트란스 바이칼 기후 라고도 한다. 여름에는 온도가 비교적 높고, 겨울에는 맑은 날씨가 지속되고 약한 바람이 분다. 복사냉각 때문에 매우 쌀쌀한 것이 특징이다. 또 여름에는 계절풍의 영향으로 비가 많이 내린다. 겨울은 여름에 비해 매우 긴 편이다. 기온의 연교차가 매우 크게 나타나는 대륙성 기후가 짙게 나타난다.\n\n냉대 동계 소우 기후는 매우 추운 기후로, 농작물을 재배할 수 있는 곳과 재배 수종 및 수가 제한되어 있다. 수수, 옥수수, 봄밀 등을 재배한다. 북부는 타이가 기후와 같이 타이가가 많이 나타나 임업활동을 한다. 숲 또한 냉대 습윤 기후와 비슷하여 남부에는 침엽수와 활엽수가 함께 나타나는 혼합림이, 북부에는 타이가(침엽수림)이 잘 나타난다. 북부는 추위에도 잘 견디는 가늘고 뾰족한 잎을 가진 침엽수가 자라기 유리하기 때문에 타이가가, 남부에는 북부보다 따뜻하여 혼합림이 잘 나타난다. 남부 지역의 극소수에서만 벼 재배가 가능하다.\n\n토양은 주로 포드졸이 나타나고, 혼합림에서는 갈색 삼림토가 나타나는데, 포드졸은 유기물이 분해되지는 않았으나, 소금기를 잘 머금지 않고 침엽수림의 특성 때문에 농작물을 재배하는 데에는 적합하지 않다. 그에 반해 갈색 삼림토는 물을 잘 흡수하고 포드졸에 비해 약산성을 띠기 때문에 농작물 재배와 목축에 알맞다.\n\n이런 냉대 동계 소우 기후가 나타나는 곳으로는 시베리아 내륙 동부, 중국 화베이 동부 및 둥베이, 대한민국 강원도 내륙 지역과 경기도 및 산악지대, 조선민주주의인민공화국 대부분, 캐나다 및 미국, 네팔의 일부 지역 등이 있으며, 특히 시베리아 동부는 세계의 한극을 만들어 낸다.\n\n그래서 이 기후가 냉대 기후의 탈을 쓴 사바나 기후라고 봐야 하는 수준에 이르게 되며, 겨울철에는 강수량이 적어져서 그렇게 건조한 만큼 불씨가 쉽게 일어나는 특성상 산불도 물론 일어나기가 쉬운 기후 대역에 속한다.\n\n앞으로 온난화 여파에 따라 이 기후가 사라질 가능성이 있는 지역도 생길 수도 있다. 단적인 예로 대한민국의 수도권, 강원도 등은 현재 Dwa로 분류되나 온대 기후로 바뀔 가능성이 있다.'''
question = '냉대 동계 소우 기후의 w는 무엇의 약자인가?'
generated_text = generate_answer(question, context)
print(f"Generated text: {generated_text}")

In [ ]:
context = '''한국의 웹툰.\n\n곽백수가 네이버에서 연재한 웹툰. 회사 생활의 이야기를 주축으로 하는 직장 만화로, 작가의 이전 흥행작인《트라우마》에서도 익히 이름을 알렸던 가우스 그룹#s-1.3이 아예 메인으로 다뤄진다. 따라서, 어떤 의미에서는 트라우마의 스핀오프작이라고도 볼수 있는 셈. 물론, 가우스 그룹의 막나가는 센스는 여전하다. 초기에는 《사이좋은 가우스전자》라는 이름이었지만 어느 순간 '사이좋은'이 빠지고 그냥 《가우스전자》가 되었다.\n\n기본적으로《트라우마》와 비슷한 분위기의 개그물이지만 블랙 코미디의 성격이 강하다. 게다가 주제가 주제인지라 회사원 입장에서 공감할 내용들이 많으며, 그런 독자들의 반응은 대개 '내가 웃어도 웃는 게 아니야.' 로 통일되어 있다. 그래도 불편한 진실과 블랙 코미디 사이에서 블랙 코미디 쪽으로 줄타기를 잘 하고 있다. 또한 브리핑 에피소드들을 보면 현 업계의 시류를 잘 집고 있어 작가가 에피소드 소재에 대한 준비를 열심히 한 듯한 느낌이 난다. 그렇기에 특히 회사원들, 즉 어느 정도 연령층이 높은 독자들에게 평이 상당히 좋다. 그래도 상대적으로 악역 포지션인 인물들이 있고 그렇지 않은 인물들이 있지만, 현실 기업과는 비교도 할 수 없을만큼의 훈훈함이 특징.'''
question = '《가우스전자》의 원래 제목은 무엇이었나?'
generated_text = generate_answer(question, context)
print(f"Generated text: {generated_text}")